# AutoCAD MegaIntellegence Project


In [1]:
#import win32com.client
import comtypes.client
import numpy as np
import pandas as pd
import math
import array
import time
app = comtypes.client.GetActiveObject('AutoCAD.Application')
#app = win32com.client.Dispatch("AutoCAD.Application")
activedoc = app.ActiveDocument
msp =activedoc.ModelSpace

In [2]:
# print all Block refs:
# for blk in activedoc.Blocks:
#     print(blk)

In [3]:
# set variables
df = pd.DataFrame(columns ={'device','start_att_line_X', 'end_att_line_X','start_att_line_Y', 'end_att_line_Y','first_point'})
df = df[['device','start_att_line_X', 'end_att_line_X','start_att_line_Y', 'end_att_line_Y','first_point']]
dim_aligned, dim_rotated, blockref, i = 0,0,0,0
# view each object in ModelSpace
for item in msp:
    #TODO: need to check if that actually device block
    #if it's a device:
    if item.ObjectName == 'AcDbBlockReference':
        #find BLOCK object using given BlockRef
        block = activedoc.Blocks.Item(item.Name)
        print (f'Block ref name: {block}')
        for element in block:
            if element.Layer =="attachment point":
                # fill the dataframe with the coordinates of the attachment points. Coordinates consists of distance from point (0,0,0) and insertion point
                  df = df.append({'device': i, \
                                'start_att_line_X': element.StartPoint[0] + item.InsertionPoint[0], \
                                'start_att_line_Y': element.StartPoint[1] + item.InsertionPoint[1], \
                                'end_att_line_X': element.EndPoint[0] + item.InsertionPoint[0],\
                                'end_att_line_Y': element.EndPoint[1] + item.InsertionPoint[1]},  ignore_index=True)
        i+=1
                
df['device'] = df['device'].astype(int)


Block ref name: <POINTER(IAcadBlock) ptr=0x22d1fa52778 at 22d20e75bc8>
Block ref name: <POINTER(IAcadBlock) ptr=0x22d1fa52f58 at 22d1e415ac8>
Block ref name: <POINTER(IAcadBlock) ptr=0x22d1fa40a28 at 22d1e415648>


In [4]:
# Length of attraction line must be 5mm
# Is the line in this row horisontal?
df['horizontal']= (df.iloc[:,1] == df.iloc[:,2])
# Find center of this line at axis X. If line is horizontal...
df['point_X'] =np.where(df['horizontal'], \
                        # any coordinate is suitable
                        df.iloc[:, 1], \
                        # otherwise, take the middle of the line by X
                        (np.minimum(df.iloc[:, 1], df.iloc[:, 2]) +2.5))

#same for Y
df['point_Y'] =np.where(~ df['horizontal'], \
                        df.iloc[:, 3], \
                        (np.minimum(df.iloc[:, 3], df.iloc[:, 4]) +2.5)) 
# Make feature with coordinates of attachment point
df['first_point'] =df.point_X.astype(str) + ',' + df.point_Y.astype(str)



In [5]:
#df
    #print(f'got an item: {item}')
    #print(f'Object name: {item.ObjectName}')
    #print(item._methods_)
dims = []
for item in msp:    
    if item.ObjectName =='AcDbRotatedDimension':
        print('found rotated dimention.')
        print(item)
        #print(f'Object layer name: {item.Layer}\n')       
        dim_rotated = item
    elif item.ObjectName == 'AcDbAlignedDimension':
        print('found aligned dimention\n')
        dim_aligned = item
        dims.append(dim_aligned)
   

found aligned dimention

found aligned dimention

found aligned dimention

found aligned dimention

found rotated dimention.
<POINTER(IAcadDimRotated) ptr=0x22d1fa3fd78 at 22d1e43f8c8>
found rotated dimention.
<POINTER(IAcadDimRotated) ptr=0x22d1d757cb8 at 22d1e43fc48>


In [6]:
# set variables
df_dimensions = pd.DataFrame(columns ={'dim_No','start_X','start_Y','end_X', 'end_Y','text_pos_X','text_pos_Y','first_point','second_point'})
df_dimensions = df_dimensions[['dim_No','start_X','start_Y','end_X','end_Y','text_pos_X','text_pos_Y','first_point','second_point']]
i=0

#make the dataframe with dimensions data
for dim in dims:
    df_dimensions = df_dimensions.append({'dim_No': i+1 , \
                              'start_X': dim.ExtLine1Point[0],
                              'start_Y': dim.ExtLine1Point[1],
                              'end_X': dim.ExtLine2Point[0],
                              'end_Y': dim.ExtLine2Point[1], 
                              'text_pos_X': dim.TextPosition[0], 
                              'text_pos_Y': dim.TextPosition[1],
                              'first_point': f'{dim.ExtLine1Point[0]},{dim.ExtLine1Point[1]}',
                              'second_point': f'{dim.ExtLine2Point[0]},{dim.ExtLine2Point[1]}'}, ignore_index=True)
    i+=1


# process dataframe
df4 = df_dimensions.drop(columns = ['start_X','start_Y','end_X','end_Y'])
df2 = pd.concat([(df4.drop(columns='second_point')),(df4.drop(columns='first_point'))],sort=False)
# make 'point' feature. Make 2 rows for each dim_No. First row stores first coordinate on 1st place. Second row stores 2nd coordinate on 1st place
points = pd.concat([df2.iloc[:,3],df2.iloc[:,4]]).dropna()
points.name = 'point'
df_dimensions = pd.concat([df2, points], axis=1) \
                    .drop(columns=['first_point', 'second_point'])

df_dimensions

# df2

# df4 = df_dimensions.drop(columns = {'start_X','start_Y','end_X','end_Y'})
# pd.merge((df4.drop(columns='second_point')),\
#           (df4.drop(columns='first_point')), \
#           how ='outer', \
#           left_on= 'first_point', \
#           right_on = 'second_point')

,dim_No,text_pos_X,text_pos_Y,point
0,1,510.100512,439.184607,"475.1005121910266,460.1081545011157"
1,2,427.585409,599.101201,"471.143810326269,539.9107245164855"
2,3,506.143810,617.375849,"471.143810326269,569.9107245164855"
3,4,615.809154,554.910725,"659.2998643756055,569.9107245164855"
0,1,510.100512,439.184607,"545.1005121910266,460.1081545011157"
1,2,427.585409,599.101201,"471.143810326269,569.9107245164855"
2,3,506.143810,617.375849,"541.143810326269,569.9107245164855"
3,4,615.809154,554.910725,"659.2998643756055,539.9107245164855"


In [7]:
df_device = df.rename(columns={'first_point':'point'})\
            .drop(columns=['start_att_line_X','end_att_line_X','start_att_line_Y','end_att_line_Y',\
                           'point_X','point_Y', 'horizontal']).drop_duplicates(subset= 'point')


In [8]:
df= pd.merge(left = df_device, right = df_dimensions, how= 'outer', on='point')
df= df.fillna(0)
df['dim_No'] = df['dim_No'].astype(int)
df

,device,point,dim_No,text_pos_X,text_pos_Y
0,0,"471.143810326269,569.9107245164855",3,506.143810,617.375849
1,0,"471.143810326269,569.9107245164855",2,427.585409,599.101201
2,0,"541.143810326269,569.9107245164855",3,506.143810,617.375849
3,0,"471.143810326269,539.9107245164855",2,427.585409,599.101201
4,0,"541.143810326269,539.9107245164855",0,0.000000,0.000000
5,1,"659.2998643756055,539.9107245164855",4,615.809154,554.910725
6,1,"659.2998643756055,569.9107245164855",4,615.809154,554.910725
7,2,"475.1005121910266,460.1081545011157",1,510.100512,439.184607
8,2,"545.1005121910266,460.1081545011157",1,510.100512,439.184607


In [13]:
sset = activedoc.SelectionSets.Add('ssSSET')

In [11]:
sset.Clear()

NameError: name 'sset' is not defined

In [ ]:
#activedoc.SendCommand("(cdr (assoc 13 (entget (handent " & """" & returnObj.Handle & """" & "))))" & vbCr)

In [ ]:
#dont working
#sset.AddItems(msp.Item(dim_rotated.Handle))
#xtypeOut,xdataOut =0,0
#xtypeOut,xdataOut = dim_rotated.GetXData('')
#print(dim_rotated.GetXData(''))
#aa = dim_rotated.GetXData('')
#aa

In [14]:
#Make DXF file to parse 
filename= 'C:\\Users\\MSI\\Documents\\AutoCad_Project\\testfile'
activedoc.Export(filename, 'DXF', sset)

0

In [53]:
%%time
#count dimensions in dxf. 
#TODO: need make a function
dxf = open(filename + '.dxf', 'r')
dic={}
i,key=0,0


for line in dxf:
    if line=='DIMENSION\n':
        dic[key] = i
        print('line '+ str(i) +' had a dimension')
        key+=1        
    i+=1
   

 #dict=[line for line in dxf if line=='DIMENSION\n']
#print(dict)

#data = dxf.readlines()
#print(data)

#data = dxf.readlines(70000)

dxf.close()

print(dic)

line 6015 had a dimension
line 6083 had a dimension
line 6151 had a dimension
line 6219 had a dimension
line 6287 had a dimension
line 6347 had a dimension
{0: 6015, 1: 6083, 2: 6151, 3: 6219, 4: 6287, 5: 6347}
Wall time: 15.6 ms


In [65]:
#get the info about each dimension
%%time
dxf = open(filename + '.dxf', 'r')
i=0
data = dxf.readlines()
for d in dic:
    print(data[dic[d]:dic[d]+60])\
    if ((data[dic[d]+58])=='AcDbRotatedDimension\n') \
    else print(data[dic[d]:dic[d]+68])
dxf.close()

['DIMENSION\n', '  5\n', '1133\n', '102\n', '{ACAD_XDICTIONARY\n', '360\n', '1140\n', '102\n', '}\n', '102\n', '{ACAD_REACTORS\n', '330\n', '114A\n', '102\n', '}\n', '330\n', '1F\n', '100\n', 'AcDbEntity\n', '  8\n', 'dimensions\n', '100\n', 'AcDbDimension\n', '280\n', '     0\n', '  2\n', '*D4\n', ' 10\n', '545.1005121910266\n', ' 20\n', '426.184607317436\n', ' 30\n', '0.0\n', ' 11\n', '510.1005121910266\n', ' 21\n', '439.184607317436\n', ' 31\n', '0.0\n', ' 70\n', '    33\n', ' 71\n', '     5\n', ' 42\n', '70.0\n', ' 73\n', '     0\n', ' 74\n', '     0\n', ' 75\n', '     0\n', '  3\n', 'Copy of ISO-25\n', '100\n', 'AcDbAlignedDimension\n', ' 13\n', '475.1005121910266\n', ' 23\n', '460.1081545011156\n', ' 33\n', '0.0\n', ' 14\n', '545.1005121910266\n', ' 24\n', '460.1081545011156\n', ' 34\n', '0.0\n', '  0\n']
['DIMENSION\n', '  5\n', '1169\n', '102\n', '{ACAD_XDICTIONARY\n', '360\n', '1177\n', '102\n', '}\n', '102\n', '{ACAD_REACTORS\n', '330\n', '1182\n', '102\n', '}\n', '330\n', '1

In [62]:
dic[i+2]

6151

In [ ]:
print(dim_aligned.ExtLine1Point[0])
print(dim_aligned.ExtLine2Point)
print(dim_aligned.TextPosition)

In [ ]:
# Довольно неочевидно, как взрывать размеры, не используя vba или lisp. Здесь надо будет добыть имя блока размера для метода explode.
# for item in msp:
#     if item.ObjectName == 'AcAlignedDimension':
#         #find BLOCK object by given BlockRef
#         activedoc.Blocks.Item(item.     )
#         #item.Explode(item.ObjectID)

#         print (item.ObjectID) 

Использовать фичу DIMATFIT 

вариант добычи точек привязки rotated размера - экспорт в dwf https://adn-cis.org/forum/index.php?topic=8448.msg31681#msg31681
И не забыть SelectionSet

In [ ]:
def FindAttachmentPoint(data):
    """This function converts given data to the coordinates of attachment points. 
    
    data: dataframe with coordinates of lines in layer 'attachment point'
    
    return
    """
    #FindAttachmentPoint(df[df['device']==device])
    print(f'data:\n{data}')
    print(data.iloc[[0]])


def MakeDimVertical(msp, point1, point2, loc):
    """This function make vertical dimension.
    
    msp:  model space
    point1, point2: points of dimention
    loc - location of dimension
    """
    pass
    

def MakeDimHorizontal(msp, point1, point2, loc):
    """This function make horizontal dimension.
    
    msp:  model space
    point1, point2: points of dimention
    loc - location of dimension
    """
    pass

def ProcessAttachmentPoints(msp, objects):
    """This function make coordinates of attachment points.
    
    msp:  model space
    objects:  lines from specified layer
    """
    pass


In [ ]:
stop
# ActiveX usage examples
pt = array.array('d', [0,0,0]) # convert to variant
point = msp.AddPoint(pt)
# Add a LINE
pt1 = array.array('d', [0.0,0.0,0]) # start point
pt2 = array.array('d', [20.0,20.0,0]) # end point
line = msp.AddLine(pt1, pt2)

In [ ]:
s1 =  array.array('d', [500.0, 800.0, 0.0])
s2 =  array.array('d', [500.0, 1200.0, 0.0])
loc = array.array('d', [400.0, 1000.0, 0.0])
new_dim = msp.AddDimAligned(s1, s2, loc)
new_dim.update()

In [ ]:
df = pd.read_excel('E://Python/RubinProject/Journal.xlsx')

In [ ]:
# import win32com.client
# acad = win32com.client.Dispatch("AutoCAD.Application")

# doc = acad.ActiveDocument   # Document object


# # iterate trough all objects (entities) in the currently opened drawing
# # and if its a BlockReference, display its attributes and some other things.
# for entity in acad.ActiveDocument.ModelSpace:
#     name = entity.EntityName
#     if name == 'AcDbBlockReference':
#         HasAttributes = entity.HasAttributes
#         if HasAttributes:
#             print(entity.Name)
#             print(entity.Layer)
#             print(entity.ObjectID)
#             for attrib in entity.GetAttributes():
#                 print("  {}: {}".format(attrib.TagString, attrib.TextString))
                
#                 # update text
#                 attrib.TextString = 'modified with python'
#                 attrib.Update()

In [ ]:
     
    #explode blocks
#    elif item.ObjectName == 'AcDbBlockReference':       
#         try:
#             print(f'Found a block.\n')
#             block = item
#             block.Explode()
#         except:
#             pass    # here we have some VARIANT bug. So just proceed.
#         i+=1

Фичи размеров в качестве таргетов:
- TextPosition (rotated размеры тоже их имеют)

Запасной вариант- взрывать все размеры, вытаскивать параметры линий, и таким образом получать координаты точек привязки размера, а заодно и координаты текста. Возможно, придётся так и сделать, чтобы обеспечить непересекаемоть размеров и других линий чертежа, типа линий габаритки

todo:

точка крепления может быть не посередине линии соответствующего слоя, а где-то на пересечении с линией прибора. Сделать проверку на небольшое отклонение и выдать сообщение чтобы поправили чертёж

Как взрывать rotated размеры для извлечения координат точек - неочевидно. Xdata добыть заморочно, но зато там сразу готовые данные размера. Стоит начать с xdata